# Initial Data Exploration

This notebook provides an initial exploration of the customer churn dataset to understand:
- Data structure and schema
- Missing values and data quality
- Basic statistics and distributions
- Event types and user behavior patterns

In [ ]:
import json
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

plt.style.use("default")
sns.set_palette("husl")

print("Libraries imported successfully")

## 1. Load and Examine Data Files

In [ ]:
def load_json_data(file_path):
    """Load JSON data from file, handling both single JSON objects and JSONL format"""
    data = []
    try:
        with open(file_path) as f:
            # Try to load as single JSON first
            try:
                content = json.load(f)
                if isinstance(content, list):
                    data = content
                else:
                    data = [content]
            except json.JSONDecodeError:
                # If that fails, try JSONL format (one JSON per line)
                f.seek(0)
                for line in f:
                    line = line.strip()
                    if line:
                        data.append(json.loads(line))
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
    return data


# Load both datasets
print("Loading data files...")
mini_data = load_json_data("../data/customer_churn_mini.json")
full_data = load_json_data("../data/customer_churn.json")

print(f"Mini dataset: {len(mini_data)} records")
print(f"Full dataset: {len(full_data)} records")

In [ ]:
# Convert to DataFrames
df_mini = pd.DataFrame(mini_data)
df_full = pd.DataFrame(full_data)

print("Mini Dataset Shape:", df_mini.shape)
print("Full Dataset Shape:", df_full.shape)

# Use the mini dataset for initial exploration
df = df_mini.copy()
print(f"\nUsing mini dataset for exploration: {df.shape}")

## 2. Data Schema and Structure Analysis

In [ ]:
# Examine data structure
print("=== DATA SCHEMA ===\n")
print("Column names and data types:")
print(df.dtypes)
print(f"\nDataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

In [ ]:
# Show first few records
print("=== FIRST 3 RECORDS ===\n")
for i in range(min(3, len(df))):
    print(f"Record {i+1}:")
    print(df.iloc[i].to_dict())
    print("-" * 50)

In [ ]:
# Examine unique values for key categorical columns
categorical_cols = ["page", "auth", "method", "level", "gender"]

print("=== CATEGORICAL COLUMNS ANALYSIS ===\n")
for col in categorical_cols:
    if col in df.columns:
        print(f"{col.upper()}:")
        print(f"  Unique values: {df[col].nunique()}")

        # For EDA, we want to see NaN patterns, not hide them
        # Count NaN values separately to understand data quality
        nan_count = df[col].isnull().sum()
        total_count = len(df)
        nan_percentage = (nan_count / total_count) * 100

        if nan_count > 0:
            print(f"  Missing values: {nan_count} ({nan_percentage:.2f}%)")

        # Get unique values excluding NaN for sorting (to avoid TypeError)
        # But we'll report the missing values separately above
        unique_vals_no_nan = df[col].dropna().unique()

        try:
            sorted_vals = sorted(unique_vals_no_nan)
        except TypeError:
            # Fallback: convert to string if mixed types still cause issues
            sorted_vals = sorted([str(x) for x in unique_vals_no_nan])

        # Show non-missing unique values
        print(f"  Non-missing values: {sorted_vals}")

        # Value counts include NaN by default, which is what we want for EDA
        # This shows us the complete picture including missing data patterns
        print("  Value counts (including missing):")
        value_counts = df[col].value_counts(dropna=False).head(10)
        print(value_counts)

        # Add interpretation comments for different column types
        if col == "page":
            if nan_count > 0:
                print(
                    "     WARNING: 'page' should never be missing - indicates data quality issues"
                )
        elif col == "level":
            if nan_count > 0:
                print(
                    "     INFO: Missing subscription level might indicate guest users or data capture gaps"
                )
        elif col == "gender":
            if nan_count > 0:
                print(
                    "     INFO: Missing gender is common - users may not provide demographic info"
                )
        elif col in ["auth", "method"]:
            if nan_count > 0:
                print(
                    f"     WARNING: '{col}' should typically be present for all requests"
                )

        print("-" * 40)

## 3. Missing Values and Data Quality Analysis

In [ ]:
# Missing values analysis with visualization
print("=== MISSING VALUES ANALYSIS ===\n")
missing_info = pd.DataFrame(
    {
        "Column": df.columns,
        "Missing_Count": df.isnull().sum(),
        "Missing_Percentage": (df.isnull().sum() / len(df)) * 100,
        "Data_Type": df.dtypes,
    }
)

missing_info = missing_info.sort_values("Missing_Percentage", ascending=False)
print(missing_info)

# Highlight columns with significant missing values
high_missing = missing_info[missing_info["Missing_Percentage"] > 10]
if not high_missing.empty:
    print("\n   Columns with >10% missing values:")
    print(high_missing[["Column", "Missing_Percentage"]])

# Visualize missing values
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Missing values heatmap
missing_cols = missing_info[missing_info["Missing_Count"] > 0]["Column"].head(10)
if len(missing_cols) > 0:
    missing_matrix = df[missing_cols].isnull()
    sns.heatmap(
        missing_matrix,
        cbar=True,
        ax=axes[0],
        yticklabels=False,
        cmap="viridis",
        alpha=0.8,
    )
    axes[0].set_title("Missing Values Pattern (Sample of Data)")
    axes[0].set_xlabel("Columns with Missing Values")
else:
    axes[0].text(
        0.5,
        0.5,
        "No Missing Values Found",
        ha="center",
        va="center",
        transform=axes[0].transAxes,
        fontsize=14,
    )
    axes[0].set_title("Missing Values Pattern")

# Missing values bar chart
missing_counts = missing_info[missing_info["Missing_Count"] > 0].sort_values(
    "Missing_Percentage", ascending=True
)
if not missing_counts.empty:
    missing_counts.plot(
        x="Column",
        y="Missing_Percentage",
        kind="barh",
        ax=axes[1],
        color="salmon",
        alpha=0.7,
    )
    axes[1].set_title("Missing Values Percentage by Column")
    axes[1].set_xlabel("Missing Percentage (%)")
    axes[1].set_ylabel("Columns")
else:
    axes[1].text(
        0.5,
        0.5,
        "No Missing Values Found",
        ha="center",
        va="center",
        transform=axes[1].transAxes,
        fontsize=14,
    )
    axes[1].set_title("Missing Values Percentage")

plt.tight_layout()
plt.show()

# Missing value patterns analysis
print("\n=== MISSING VALUE PATTERNS ===")
if len(missing_cols) > 1:
    # Check if missing values occur together
    missing_patterns = df[missing_cols].isnull().value_counts()
    print("Top missing value patterns:")
    print(missing_patterns.head())
else:
    print("Insufficient columns with missing values for pattern analysis")

In [ ]:
# Data quality checks
print("=== DATA QUALITY CHECKS ===\n")

# Check for duplicate records
duplicates = df.duplicated().sum()
print(f"Duplicate records: {duplicates} ({duplicates/len(df)*100:.2f}%)")

# Check timestamp format and range
if "ts" in df.columns:
    print("\nTimestamp analysis:")
    print(f"  Min timestamp: {df['ts'].min()}")
    print(f"  Max timestamp: {df['ts'].max()}")

    # Convert to datetime for better understanding
    df["datetime"] = pd.to_datetime(df["ts"], unit="ms")
    print(f"  Date range: {df['datetime'].min()} to {df['datetime'].max()}")
    print(f"  Time span: {(df['datetime'].max() - df['datetime'].min()).days} days")

# Check user ID consistency
if "userId" in df.columns:
    print("\nUser ID analysis:")
    print(f"  Unique users: {df['userId'].nunique()}")
    print(f"  Total events: {len(df)}")
    print(f"  Avg events per user: {len(df) / df['userId'].nunique():.1f}")

## 4. Basic Statistics and Distributions

In [ ]:
# Numerical columns analysis
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("=== NUMERICAL COLUMNS STATISTICS ===\n")
print(f"Numerical columns: {numerical_cols}")

if numerical_cols:
    print("\nDescriptive statistics:")
    print(df[numerical_cols].describe())

In [ ]:
# Visualize distributions of key numerical columns with better formatting
if numerical_cols:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    # Focus on key columns for better visualization
    key_cols = ["itemInSession", "length", "sessionId", "status"]

    for i, col in enumerate(key_cols):
        if i < len(axes) and col in df.columns:
            # Handle missing values by dropping them for visualization
            data_clean = df[col].dropna()

            if col == "status":
                # Status codes as categorical
                status_counts = data_clean.value_counts()
                status_counts.plot(kind="bar", ax=axes[i], alpha=0.7, color="skyblue")
                axes[i].set_title(f"Distribution of {col}")
                axes[i].set_ylabel("Count")
                axes[i].tick_params(axis="x", rotation=45)
            else:
                # Regular histogram for continuous variables
                data_clean.hist(
                    bins=50,
                    ax=axes[i],
                    alpha=0.7,
                    color="lightgreen",
                    edgecolor="black",
                )
                axes[i].set_title(f"Distribution of {col}")
                axes[i].set_xlabel(col)
                axes[i].set_ylabel("Frequency")

                # Add mean line
                mean_val = data_clean.mean()
                axes[i].axvline(
                    mean_val,
                    color="red",
                    linestyle="--",
                    alpha=0.7,
                    label=f"Mean: {mean_val:.1f}",
                )
                axes[i].legend()

    # User activity distribution
    if "userId" in df.columns:
        user_events = df.groupby("userId").size()
        user_events.hist(
            bins=30, ax=axes[4], alpha=0.7, color="orange", edgecolor="black"
        )
        axes[4].set_title("Events per User Distribution")
        axes[4].set_xlabel("Number of Events")
        axes[4].set_ylabel("Number of Users")

    # Song length analysis (excluding missing)
    if "length" in df.columns:
        length_clean = df["length"].dropna()
        if len(length_clean) > 0:
            # Convert to minutes for better interpretation
            length_minutes = length_clean / 60
            length_minutes.hist(
                bins=50, ax=axes[5], alpha=0.7, color="purple", edgecolor="black"
            )
            axes[5].set_title("Song Length Distribution")
            axes[5].set_xlabel("Song Length (minutes)")
            axes[5].set_ylabel("Frequency")
            axes[5].axvline(
                length_minutes.mean(),
                color="red",
                linestyle="--",
                alpha=0.7,
                label=f"Mean: {length_minutes.mean():.1f} min",
            )
            axes[5].legend()

    plt.tight_layout()
    plt.show()

## 5. Event Types and User Behavior Patterns

In [ ]:
# Event types analysis with improved visualizations
print("=== EVENT TYPES ANALYSIS ===\n")

if "page" in df.columns:
    event_counts = df["page"].value_counts()
    print("Event type distribution:")
    print(event_counts)

    # Create subplots for better visualization
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))

    # Top 10 events bar chart
    top_events = event_counts.head(10)
    top_events.plot(kind="bar", ax=axes[0, 0], color="steelblue", alpha=0.8)
    axes[0, 0].set_title("Top 10 Event Types Distribution")
    axes[0, 0].set_xlabel("Event Type")
    axes[0, 0].set_ylabel("Count")
    axes[0, 0].tick_params(axis="x", rotation=45)

    # Pie chart for TOP 5 events only (to avoid overcrowding)
    top_5_events = event_counts.head(5)
    other_events_count = event_counts.iloc[5:].sum()

    # Create data for pie chart with "Others" category
    pie_data = top_5_events.copy()
    pie_data["Others"] = other_events_count

    # Create pie chart with better spacing and readable labels
    wedges, texts, autotexts = axes[0, 1].pie(
        pie_data.values,
        labels=pie_data.index,
        autopct="%1.1f%%",
        startangle=90,
        colors=sns.color_palette("Set2", len(pie_data)),
        textprops={"fontsize": 10},
    )

    # Position labels outside the pie to avoid overlap
    for text in texts:
        text.set_fontsize(9)

    # Move percentage labels further out
    for autotext in autotexts:
        autotext.set_color("white")
        autotext.set_fontweight("bold")
        autotext.set_fontsize(9)

    axes[0, 1].set_title(
        "Top 5 Events Distribution\n(Others = Remaining 17 Events Combined)",
        fontsize=11,
        pad=15,
    )

    # All events horizontal bar chart (sorted)
    event_counts.plot(kind="barh", ax=axes[1, 0], color="lightcoral", alpha=0.8)
    axes[1, 0].set_title("All Event Types (Complete List)")
    axes[1, 0].set_xlabel("Count")
    axes[1, 0].set_ylabel("Event Type")

    # Log scale for better visualization of small events
    event_counts.plot(
        kind="bar", ax=axes[1, 1], color="darkgreen", alpha=0.8, logy=True
    )
    axes[1, 1].set_title("Event Types Distribution (Log Scale)")
    axes[1, 1].set_xlabel("Event Type")
    axes[1, 1].set_ylabel("Count (Log Scale)")
    axes[1, 1].tick_params(axis="x", rotation=90)

    plt.tight_layout()
    plt.show()

    # Calculate percentages
    event_pct = (event_counts / event_counts.sum()) * 100
    print("\nEvent type percentages:")
    for event, pct in event_pct.head(10).items():
        print(f"  {event}: {pct:.2f}%")

    # Event categories analysis
    print("\n=== EVENT CATEGORIZATION ===")
    music_events = ["NextSong", "Thumbs Up", "Thumbs Down", "Add to Playlist"]
    navigation_events = ["Home", "About", "Help", "Settings", "Save Settings"]
    auth_events = ["Login", "Logout", "Register", "Submit Registration"]
    subscription_events = [
        "Upgrade",
        "Submit Upgrade",
        "Downgrade",
        "Submit Downgrade",
        "Cancel",
        "Cancellation Confirmation",
    ]
    social_events = ["Add Friend"]
    other_events = ["Roll Advert", "Error"]

    categories = {
        "Music Interaction": music_events,
        "Navigation": navigation_events,
        "Authentication": auth_events,
        "Subscription": subscription_events,
        "Social": social_events,
        "Other": other_events,
    }

    category_counts = {}
    for category, events in categories.items():
        count = event_counts[event_counts.index.isin(events)].sum()
        category_counts[category] = count
        print(f"  {category}: {count:,} events ({count/event_counts.sum()*100:.1f}%)")

    # Visualize event categories with clean layout
    plt.figure(figsize=(14, 6))

    # Bar chart for event categories (more readable than pie for 6 categories)
    plt.subplot(1, 2, 1)
    category_series = pd.Series(category_counts)
    bars = category_series.plot(
        kind="bar", color=sns.color_palette("Set2", len(category_counts)), alpha=0.8
    )
    plt.title("Event Categories Distribution", fontsize=14)
    plt.xlabel("Category")
    plt.ylabel("Number of Events")
    plt.xticks(rotation=45, ha="right")

    # Add value labels on bars
    for i, bar in enumerate(bars.patches):
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + height * 0.01,
            f"{int(height):,}",
            ha="center",
            va="bottom",
            fontsize=9,
        )

    # Pie chart for subscription level breakdown (more meaningful)
    if "level" in df.columns:
        plt.subplot(1, 2, 2)
        level_counts = df["level"].value_counts(dropna=True)
        wedges, texts, autotexts = plt.pie(
            level_counts.values,
            labels=level_counts.index,
            autopct="%1.1f%%",
            startangle=90,
            colors=["#ff9999", "#66b3ff"],
            textprops={"fontsize": 12},
        )

        for autotext in autotexts:
            autotext.set_color("white")
            autotext.set_fontweight("bold")
            autotext.set_fontsize(11)

        plt.title("User Subscription Distribution", fontsize=14)

    plt.tight_layout()
    plt.show()

In [ ]:
# User activity patterns
print("=== USER ACTIVITY PATTERNS ===\n")

if "userId" in df.columns:
    user_activity = (
        df.groupby("userId")
        .agg(
            {
                "page": "count",  # Total events per user
                "sessionId": "nunique",  # Unique sessions per user
                "ts": ["min", "max"],  # Activity time range
            }
        )
        .reset_index()
    )

    user_activity.columns = [
        "userId",
        "total_events",
        "unique_sessions",
        "first_event",
        "last_event",
    ]

    # Calculate activity duration in days
    user_activity["activity_duration_days"] = (
        user_activity["last_event"] - user_activity["first_event"]
    ) / (1000 * 60 * 60 * 24)
    user_activity["events_per_session"] = (
        user_activity["total_events"] / user_activity["unique_sessions"]
    )

    print("User activity statistics:")
    print(
        user_activity[
            [
                "total_events",
                "unique_sessions",
                "activity_duration_days",
                "events_per_session",
            ]
        ].describe()
    )

    # Visualize user activity distribution
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    user_activity["total_events"].hist(bins=30, ax=axes[0, 0], alpha=0.7)
    axes[0, 0].set_title("Distribution of Total Events per User")
    axes[0, 0].set_xlabel("Total Events")

    user_activity["unique_sessions"].hist(bins=30, ax=axes[0, 1], alpha=0.7)
    axes[0, 1].set_title("Distribution of Sessions per User")
    axes[0, 1].set_xlabel("Unique Sessions")

    user_activity["activity_duration_days"].hist(bins=30, ax=axes[1, 0], alpha=0.7)
    axes[1, 0].set_title("Distribution of Activity Duration (Days)")
    axes[1, 0].set_xlabel("Days")

    user_activity["events_per_session"].hist(bins=30, ax=axes[1, 1], alpha=0.7)
    axes[1, 1].set_title("Distribution of Events per Session")
    axes[1, 1].set_xlabel("Events per Session")

    plt.tight_layout()
    plt.show()

In [ ]:
# Subscription level analysis - REMOVED REDUNDANT PIE CHART
if "level" in df.columns:
    print("=== SUBSCRIPTION LEVEL ANALYSIS ===\n")

    # Get value counts excluding NaN for cleaner analysis
    level_dist = df["level"].value_counts(
        dropna=False
    )  # Include NaN to see missing data
    print("Subscription level distribution:")
    print(level_dist)

    # Check for missing values and report
    missing_level_count = df["level"].isnull().sum()
    if missing_level_count > 0:
        print(
            f"\n  Note: {missing_level_count} records have missing subscription level"
        )

    # Events by subscription level (exclude NaN for cleaner pivot)
    df_no_missing_level = df.dropna(subset=["level"])  # Only for this analysis
    level_events = (
        df_no_missing_level.groupby(["level", "page"]).size().unstack(fill_value=0)
    )

    print("\nDIAGNOSTIC: Full event breakdown by subscription level:")
    print(f"Total event types in data: {level_events.shape[1]}")
    print(
        f"Event types that appear for both paid and free users: {(level_events > 0).all().sum()}"
    )
    print(
        f"Event types that appear for at least one subscription level: {(level_events > 0).any().sum()}"
    )

    print("\nTop events by subscription level (showing top 10 overall):")
    top_events_overall = level_events.sum().nlargest(10)
    print("Overall event ranking:")
    print(top_events_overall)

    print("\nTop 10 events broken down by subscription level:")
    top_10_events_df = level_events.loc[:, top_events_overall.index]
    print(top_10_events_df)

    # Single focused visualization - Events by subscription level comparison
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Bar chart - Show top 5 events that have activity in both subscription levels
    # Filter to events that have non-zero counts for both paid and free
    events_with_both_levels = level_events.loc[:, (level_events > 0).all()]

    if not events_with_both_levels.empty and events_with_both_levels.shape[1] >= 3:
        # Take top 5 events that appear in both levels
        top_events_both_levels = events_with_both_levels.sum().nlargest(5)
        events_to_plot = level_events.loc[:, top_events_both_levels.index]

        events_to_plot.plot(kind="bar", ax=axes[0])
        axes[0].set_title(
            f"Top {len(top_events_both_levels)} Events by Subscription Level\n(Events present in both paid & free)",
            fontsize=12,
        )
        axes[0].set_xlabel("Subscription Level")
        axes[0].set_ylabel("Event Count")
        axes[0].tick_params(
            axis="x", rotation=0
        )  # Keep subscription level labels horizontal
        axes[0].legend(title="Event Type", bbox_to_anchor=(1.05, 1), loc="upper left")
        axes[0].grid(axis="y", alpha=0.3)

        print("\nCHART EXPLANATION:")
        print(
            f"Showing {len(top_events_both_levels)} event types that have activity in BOTH paid and free users"
        )
        print(f"These are: {list(top_events_both_levels.index)}")

    else:
        # Fallback: show top events even if they don't appear in both levels
        if not level_events.empty:
            available_events = min(level_events.shape[1], 5)
            top_available = level_events.sum().nlargest(available_events)
            level_events.loc[:, top_available.index].plot(kind="bar", ax=axes[0])
            axes[0].set_title(
                f"Top {available_events} Events by Subscription Level\n(May not appear in both levels)",
                fontsize=12,
            )
            axes[0].set_xlabel("Subscription Level")
            axes[0].set_ylabel("Event Count")
            axes[0].tick_params(axis="x", rotation=0)
            axes[0].legend(
                title="Event Type", bbox_to_anchor=(1.05, 1), loc="upper left"
            )
            axes[0].grid(axis="y", alpha=0.3)

            print("\n📊 CHART EXPLANATION:")
            print(
                f"Showing top {available_events} events overall (some may not appear in both subscription levels)"
            )
        else:
            axes[0].text(
                0.5,
                0.5,
                "Insufficient data for event analysis",
                ha="center",
                va="center",
                transform=axes[0].transAxes,
            )
            axes[0].set_title("Events by Subscription Level")

    # Subscription behavior comparison - Show event diversity by level
    if not level_events.empty:
        # Calculate event diversity (number of different event types) per subscription level
        event_diversity = (level_events > 0).sum(axis=1)

        ax = axes[1]
        bars = event_diversity.plot(
            kind="bar", ax=ax, color=["#ff9999", "#66b3ff"], alpha=0.8
        )
        ax.set_title("Event Type Diversity by Subscription Level", fontsize=12)
        ax.set_xlabel("Subscription Level")
        ax.set_ylabel("Number of Different Event Types Used")
        ax.tick_params(axis="x", rotation=0)
        ax.grid(axis="y", alpha=0.3)

        # Add value labels on bars
        for bar in bars.patches:
            height = bar.get_height()
            ax.text(
                bar.get_x() + bar.get_width() / 2.0,
                height + height * 0.01,
                f"{int(height)}",
                ha="center",
                va="bottom",
                fontsize=11,
            )

        print("\nEVENT DIVERSITY ANALYSIS:")
        for level, diversity in event_diversity.items():
            total_events = level_events.loc[level].sum()
            print(
                f"  {level.upper()} users: Use {diversity} different event types (Total: {total_events:,} events)"
            )

    plt.tight_layout()
    plt.show()

    # Additional analysis: Show events that are unique to each subscription level
    if not level_events.empty:
        print("\nADDITIONAL INSIGHTS:")

        # Events exclusive to paid users
        paid_only_events = level_events.loc["paid"][level_events.loc["paid"] > 0]
        free_only_events = level_events.loc["free"][level_events.loc["free"] > 0]

        paid_exclusive = paid_only_events[level_events.loc["free"] == 0]
        free_exclusive = free_only_events[level_events.loc["paid"] == 0]

        if not paid_exclusive.empty:
            print(f"\nEvents exclusive to PAID users: {list(paid_exclusive.index)}")
            print(paid_exclusive.head())
        else:
            print("\nNo events are exclusive to PAID users")

        if not free_exclusive.empty:
            print(f"\nEvents exclusive to FREE users: {list(free_exclusive.index)}")
            print(free_exclusive.head())
        else:
            print("\nNo events are exclusive to FREE users")

        # Event distribution percentages by level
        print("\nEvent distribution percentages by subscription level:")
        level_event_pct = level_events.div(level_events.sum(axis=1), axis=0) * 100
        print(level_event_pct.loc[:, top_events_overall.head().index].round(2))

## 6. Temporal Patterns Analysis

In [ ]:
# Enhanced temporal patterns analysis
if "datetime" in df.columns:
    print("=== TEMPORAL PATTERNS ===\n")

    # Add temporal features
    df["hour"] = df["datetime"].dt.hour
    df["day_of_week"] = df["datetime"].dt.dayofweek
    df["date"] = df["datetime"].dt.date

    # Hourly activity pattern
    hourly_activity = df["hour"].value_counts().sort_index()
    print("Activity by hour of day:")
    print(hourly_activity)

    # Daily activity pattern
    daily_activity = df["day_of_week"].value_counts().sort_index()
    day_names = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    print("\nActivity by day of week:")
    for day_num, count in daily_activity.items():
        print(f"  {day_names[day_num]}: {count}")

    # Enhanced temporal visualizations
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))

    # Hourly pattern with style improvements
    hourly_activity.plot(
        kind="bar", ax=axes[0, 0], color="skyblue", alpha=0.8, edgecolor="black"
    )
    axes[0, 0].set_title(
        "Activity Distribution by Hour of Day", fontsize=14, fontweight="bold"
    )
    axes[0, 0].set_xlabel("Hour of Day")
    axes[0, 0].set_ylabel("Number of Events")
    axes[0, 0].tick_params(axis="x", rotation=0)
    axes[0, 0].grid(axis="y", alpha=0.3)

    # Daily pattern with day names
    daily_activity_named = pd.Series(
        daily_activity.values, index=[day_names[i] for i in daily_activity.index]
    )
    daily_activity_named.plot(
        kind="bar", ax=axes[0, 1], color="lightgreen", alpha=0.8, edgecolor="black"
    )
    axes[0, 1].set_title(
        "Activity Distribution by Day of Week", fontsize=14, fontweight="bold"
    )
    axes[0, 1].set_xlabel("Day of Week")
    axes[0, 1].set_ylabel("Number of Events")
    axes[0, 1].tick_params(axis="x", rotation=45)
    axes[0, 1].grid(axis="y", alpha=0.3)

    # Activity timeline with smoothing
    daily_timeline = df.groupby("date").size()
    daily_timeline.index = pd.to_datetime(daily_timeline.index)
    daily_timeline.plot(ax=axes[1, 0], color="orange", alpha=0.8, linewidth=2)
    axes[1, 0].set_title("Daily Activity Timeline", fontsize=14, fontweight="bold")
    axes[1, 0].set_xlabel("Date")
    axes[1, 0].set_ylabel("Daily Event Count")
    axes[1, 0].grid(alpha=0.3)

    # Add 7-day rolling average
    rolling_avg = daily_timeline.rolling(window=7, center=True).mean()
    rolling_avg.plot(
        ax=axes[1, 0],
        color="red",
        linestyle="--",
        alpha=0.7,
        linewidth=2,
        label="7-day Moving Average",
    )
    axes[1, 0].legend()

    # Session length distribution with better binning
    if "sessionId" in df.columns:
        session_lengths = df.groupby(["userId", "sessionId"]).size()
        session_lengths.hist(
            bins=50, ax=axes[1, 1], alpha=0.7, color="purple", edgecolor="black"
        )
        axes[1, 1].set_title(
            "Session Length Distribution", fontsize=14, fontweight="bold"
        )
        axes[1, 1].set_xlabel("Events per Session")
        axes[1, 1].set_ylabel("Number of Sessions")
        axes[1, 1].axvline(
            session_lengths.mean(),
            color="red",
            linestyle="--",
            label=f"Mean: {session_lengths.mean():.1f}",
        )
        axes[1, 1].axvline(
            session_lengths.median(),
            color="green",
            linestyle="--",
            label=f"Median: {session_lengths.median():.1f}",
        )
        axes[1, 1].legend()
        axes[1, 1].grid(axis="y", alpha=0.3)

    # Heatmap of activity by hour and day
    if len(df) > 1000:  # Only if we have enough data
        hour_day_activity = (
            df.groupby(["day_of_week", "hour"]).size().unstack(fill_value=0)
        )
        hour_day_activity.index = [day_names[i] for i in hour_day_activity.index]

        sns.heatmap(
            hour_day_activity,
            ax=axes[2, 0],
            cmap="YlOrRd",
            cbar_kws={"label": "Number of Events"},
            annot=False,
        )
        axes[2, 0].set_title(
            "Activity Heatmap: Hour vs Day of Week", fontsize=14, fontweight="bold"
        )
        axes[2, 0].set_xlabel("Hour of Day")
        axes[2, 0].set_ylabel("Day of Week")

    # User activity over time (sample of most active users)
    if "userId" in df.columns:
        top_users = df["userId"].value_counts().head(5).index
        user_timeline = (
            df[df["userId"].isin(top_users)]
            .groupby(["date", "userId"])
            .size()
            .unstack(fill_value=0)
        )
        user_timeline.index = pd.to_datetime(user_timeline.index)

        for user in top_users:
            if user in user_timeline.columns:
                user_timeline[user].plot(
                    ax=axes[2, 1], alpha=0.7, linewidth=2, label=f"User {user}"
                )

        axes[2, 1].set_title(
            "Top 5 Users Activity Timeline", fontsize=14, fontweight="bold"
        )
        axes[2, 1].set_xlabel("Date")
        axes[2, 1].set_ylabel("Daily Events")
        axes[2, 1].legend(bbox_to_anchor=(1.05, 1), loc="upper left")
        axes[2, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Statistical summary of temporal patterns
    print("\n=== TEMPORAL INSIGHTS ===")
    peak_hour = hourly_activity.idxmax()
    peak_day = day_names[daily_activity.idxmax()]
    print(f"Peak activity hour: {peak_hour}:00")
    print(f"Peak activity day: {peak_day}")
    print(f"Average daily events: {daily_timeline.mean():.1f}")
    print(
        f"Most active date: {daily_timeline.idxmax()} ({daily_timeline.max()} events)"
    )
    print(
        f"Least active date: {daily_timeline.idxmin()} ({daily_timeline.min()} events)"
    )

## 7. Potential Churn Indicators

In [ ]:
# Correlation analysis of numerical features
print("=== CORRELATION ANALYSIS ===\n")

# Select numerical columns for correlation analysis
numerical_cols_for_corr = [
    "ts",
    "sessionId",
    "status",
    "itemInSession",
    "registration",
    "length",
]
existing_num_cols = [col for col in numerical_cols_for_corr if col in df.columns]

if len(existing_num_cols) > 1:
    # Calculate correlation matrix
    correlation_matrix = df[existing_num_cols].corr()

    print("Correlation Matrix:")
    print(correlation_matrix.round(3))

    # Visualize correlation matrix
    plt.figure(figsize=(12, 10))

    # Create heatmap
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
    sns.heatmap(
        correlation_matrix,
        mask=mask,
        annot=True,
        cmap="RdYlBu_r",
        center=0,
        square=True,
        fmt=".2f",
        cbar_kws={"label": "Correlation Coefficient"},
    )
    plt.title(
        "Correlation Matrix of Numerical Features", fontsize=16, fontweight="bold"
    )
    plt.tight_layout()
    plt.show()

    # Identify strong correlations
    print("\n=== STRONG CORRELATIONS (>0.5 or <-0.5) ===")
    strong_corr_pairs = []

    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if abs(corr_val) > 0.5:
                strong_corr_pairs.append(
                    (
                        correlation_matrix.columns[i],
                        correlation_matrix.columns[j],
                        corr_val,
                    )
                )

    if strong_corr_pairs:
        for col1, col2, corr in strong_corr_pairs:
            print(f"  {col1} <-> {col2}: {corr:.3f}")
    else:
        print("  No strong correlations found between numerical features")

# Analyze relationship between numerical features and categorical targets
print("\n=== FEATURE RELATIONSHIPS WITH CATEGORICAL VARIABLES ===")

# Subscription level vs numerical features
if "level" in df.columns and existing_num_cols:
    print("\nNumerical features by subscription level:")

    # Create comparison plots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()

    # Select key numerical features for comparison
    key_features = ["itemInSession", "length", "sessionId"]

    for i, feature in enumerate(key_features):
        if feature in df.columns and i < len(axes):
            # Box plot comparison
            df_clean = df.dropna(subset=[feature, "level"])
            if len(df_clean) > 0:
                df_clean.boxplot(column=feature, by="level", ax=axes[i])
                axes[i].set_title(f"{feature.title()} by Subscription Level")
                axes[i].set_xlabel("Subscription Level")
                axes[i].set_ylabel(feature.title())

    # User activity comparison
    if "userId" in df.columns:
        user_level_activity = (
            df.groupby(["userId", "level"]).size().reset_index(name="activity_count")
        )
        user_level_activity.boxplot(column="activity_count", by="level", ax=axes[3])
        axes[3].set_title("User Activity by Subscription Level")
        axes[3].set_xlabel("Subscription Level")
        axes[3].set_ylabel("Activity Count")

    plt.tight_layout()
    plt.show()

    # Statistical comparison
    from scipy import stats

    print("\nStatistical tests comparing subscription levels:")
    for feature in key_features:
        if feature in df.columns:
            paid_data = df[df["level"] == "paid"][feature].dropna()
            free_data = df[df["level"] == "free"][feature].dropna()

            if len(paid_data) > 0 and len(free_data) > 0:
                # Perform t-test
                t_stat, p_value = stats.ttest_ind(paid_data, free_data)

                print(f"  {feature}:")
                print(
                    f"    Paid users - Mean: {paid_data.mean():.2f}, Std: {paid_data.std():.2f}"
                )
                print(
                    f"    Free users - Mean: {free_data.mean():.2f}, Std: {free_data.std():.2f}"
                )
                print(
                    f"    T-test p-value: {p_value:.6f} {'(Significant)' if p_value < 0.05 else '(Not significant)'}"
                )

# Feature engineering ideas based on correlations
print("\n=== FEATURE ENGINEERING INSIGHTS ===")
print("Based on correlation analysis and data patterns:")
print("1. Session-based features: itemInSession shows user engagement within sessions")
print(
    "2. Temporal features: Activity patterns vary by hour/day - useful for engagement scoring"
)
print(
    "3. User behavior ratios: Events per session, sessions per day could be predictive"
)
print(
    "4. Subscription comparison: Statistical differences suggest subscription level is important"
)
print(
    "5. Missing value patterns: Song-related missing values occur together - may indicate non-music events"
)

## 7. Correlation Analysis

In [ ]:
# Enhanced churn indicators analysis with visualizations
print("=== POTENTIAL CHURN INDICATORS ===\n")

# Search for churn-related events
if "page" in df.columns:
    churn_keywords = ["cancel", "downgrade", "unsubscrib", "logout", "error"]
    churn_events = []

    for event in df["page"].unique():
        if any(keyword in str(event).lower() for keyword in churn_keywords):
            churn_events.append(event)

    print(f"Potential churn-related events found: {churn_events}")

    if churn_events:
        churn_data = []
        for event in churn_events:
            event_df = df[df["page"] == event]
            count = event_df.shape[0]
            users = event_df["userId"].nunique()
            churn_data.append(
                {
                    "Event": event,
                    "Count": count,
                    "Users": users,
                    "Avg_per_User": count / users if users > 0 else 0,
                }
            )
            print(f"  {event}: {count} events, {users} unique users")

        churn_df = pd.DataFrame(churn_data)

        # Visualize churn events
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # Churn events count
        churn_df.set_index("Event")["Count"].plot(
            kind="bar", ax=axes[0, 0], color="red", alpha=0.7
        )
        axes[0, 0].set_title("Churn-Related Events Count")
        axes[0, 0].set_ylabel("Number of Events")
        axes[0, 0].tick_params(axis="x", rotation=45)
        axes[0, 0].grid(axis="y", alpha=0.3)

        # Users affected by churn events
        churn_df.set_index("Event")["Users"].plot(
            kind="bar", ax=axes[0, 1], color="orange", alpha=0.7
        )
        axes[0, 1].set_title("Users Affected by Churn Events")
        axes[0, 1].set_ylabel("Number of Users")
        axes[0, 1].tick_params(axis="x", rotation=45)
        axes[0, 1].grid(axis="y", alpha=0.3)

        # Average events per user for churn events
        churn_df.set_index("Event")["Avg_per_User"].plot(
            kind="bar", ax=axes[1, 0], color="darkred", alpha=0.7
        )
        axes[1, 0].set_title("Average Churn Events per User")
        axes[1, 0].set_ylabel("Events per User")
        axes[1, 0].tick_params(axis="x", rotation=45)
        axes[1, 0].grid(axis="y", alpha=0.3)

        # Churn events timeline
        if "datetime" in df.columns:
            churn_timeline = (
                df[df["page"].isin(churn_events)]
                .groupby([df["datetime"].dt.date, "page"])
                .size()
                .unstack(fill_value=0)
            )
            churn_timeline.index = pd.to_datetime(churn_timeline.index)

            for event in churn_events:
                if event in churn_timeline.columns:
                    churn_timeline[event].plot(
                        ax=axes[1, 1], alpha=0.7, linewidth=2, label=event
                    )

            axes[1, 1].set_title("Churn Events Timeline")
            axes[1, 1].set_xlabel("Date")
            axes[1, 1].set_ylabel("Daily Churn Events")
            axes[1, 1].legend()
            axes[1, 1].grid(alpha=0.3)

        plt.tight_layout()
        plt.show()

# Enhanced user engagement analysis
if "userId" in df.columns:
    print("\n=== USER ENGAGEMENT ANALYSIS ===\n")

    # Calculate comprehensive engagement metrics per user
    engagement_metrics = (
        df.groupby("userId")
        .agg(
            {
                "page": [
                    "count",
                    lambda x: x.nunique(),
                ],  # Total events and unique event types
                "sessionId": "nunique",  # Unique sessions
                "ts": lambda x: (x.max() - x.min())
                / (1000 * 60 * 60 * 24),  # Activity span in days
            }
        )
        .reset_index()
    )

    engagement_metrics.columns = [
        "userId",
        "total_events",
        "unique_event_types",
        "unique_sessions",
        "activity_span_days",
    ]
    engagement_metrics["events_per_day"] = engagement_metrics["total_events"] / (
        engagement_metrics["activity_span_days"] + 1
    )
    engagement_metrics["events_per_session"] = (
        engagement_metrics["total_events"] / engagement_metrics["unique_sessions"]
    )

    print("Engagement metrics summary:")
    print(
        engagement_metrics[
            [
                "total_events",
                "unique_event_types",
                "unique_sessions",
                "activity_span_days",
                "events_per_day",
            ]
        ].describe()
    )

    # Visualize engagement metrics
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    metrics = [
        "total_events",
        "unique_event_types",
        "unique_sessions",
        "activity_span_days",
        "events_per_day",
        "events_per_session",
    ]
    colors = ["blue", "green", "red", "orange", "purple", "brown"]

    for i, (metric, color) in enumerate(zip(metrics, colors, strict=False)):
        engagement_metrics[metric].hist(
            bins=30, ax=axes[i], alpha=0.7, color=color, edgecolor="black"
        )
        axes[i].set_title(f'Distribution of {metric.replace("_", " ").title()}')
        axes[i].set_xlabel(metric.replace("_", " ").title())
        axes[i].set_ylabel("Number of Users")
        axes[i].grid(axis="y", alpha=0.3)

        # Add mean line
        mean_val = engagement_metrics[metric].mean()
        axes[i].axvline(
            mean_val,
            color="red",
            linestyle="--",
            alpha=0.8,
            label=f"Mean: {mean_val:.1f}",
        )
        axes[i].legend()

    plt.tight_layout()
    plt.show()

    # User segmentation based on engagement
    print("\n=== USER SEGMENTATION BY ENGAGEMENT ===")

    # Define engagement tiers based on quantiles
    engagement_metrics["engagement_tier"] = pd.cut(
        engagement_metrics["events_per_day"],
        bins=[
            0,
            engagement_metrics["events_per_day"].quantile(0.25),
            engagement_metrics["events_per_day"].quantile(0.5),
            engagement_metrics["events_per_day"].quantile(0.75),
            engagement_metrics["events_per_day"].max(),
        ],
        labels=["Low", "Medium-Low", "Medium-High", "High"],
        include_lowest=True,
    )

    tier_summary = engagement_metrics["engagement_tier"].value_counts().sort_index()
    print("User engagement tiers:")
    print(tier_summary)

    # Visualize user tiers
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    tier_summary.plot(
        kind="pie", autopct="%1.1f%%", colors=sns.color_palette("viridis", 4)
    )
    plt.title("User Distribution by Engagement Tier")
    plt.ylabel("")

    # Box plot of engagement metrics by tier
    plt.subplot(2, 2, 2)
    engagement_metrics.boxplot(
        column="events_per_day", by="engagement_tier", ax=plt.gca()
    )
    plt.title("Events per Day by Engagement Tier")
    plt.xlabel("Engagement Tier")
    plt.ylabel("Events per Day")

    plt.subplot(2, 2, 3)
    engagement_metrics.boxplot(
        column="total_events", by="engagement_tier", ax=plt.gca()
    )
    plt.title("Total Events by Engagement Tier")
    plt.xlabel("Engagement Tier")
    plt.ylabel("Total Events")

    plt.subplot(2, 2, 4)
    engagement_metrics.boxplot(
        column="activity_span_days", by="engagement_tier", ax=plt.gca()
    )
    plt.title("Activity Span by Engagement Tier")
    plt.xlabel("Engagement Tier")
    plt.ylabel("Activity Span (Days)")

    plt.tight_layout()
    plt.show()

    # Identify potentially churned users (low engagement)
    low_engagement_threshold = engagement_metrics["events_per_day"].quantile(0.1)
    low_engagement_users = engagement_metrics[
        engagement_metrics["events_per_day"] <= low_engagement_threshold
    ]

    print(
        f"\nUsers with potentially low engagement (bottom 10%): {len(low_engagement_users)}"
    )
    print(f"Low engagement threshold: {low_engagement_threshold:.2f} events per day")

    # Analyze churn events by subscription level
    if "level" in df.columns and churn_events:
        print("\n=== CHURN EVENTS BY SUBSCRIPTION LEVEL ===")
        churn_by_level = (
            df[df["page"].isin(churn_events)]
            .groupby(["level", "page"])
            .size()
            .unstack(fill_value=0)
        )
        print("Churn events by subscription level:")
        print(churn_by_level)

        # Visualize churn by level
        plt.figure(figsize=(12, 6))
        churn_by_level.plot(kind="bar", stacked=True, colormap="Set3")
        plt.title("Churn Events by Subscription Level")
        plt.xlabel("Subscription Level")
        plt.ylabel("Number of Events")
        plt.legend(title="Churn Event", bbox_to_anchor=(1.05, 1), loc="upper left")
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.show()

## 8. Key Findings Summary

In [ ]:
print("=== INITIAL DATA EXPLORATION SUMMARY ===\n")

print("Dataset Overview:")
print(
    f"   • Mini dataset: {df.shape[0]:,} events from {df['userId'].nunique() if 'userId' in df.columns else 'N/A'} users"
)
print(
    f"   • Time period: {(df['datetime'].max() - df['datetime'].min()).days if 'datetime' in df.columns else 'N/A'} days"
)
print(
    f"   • Event types: {df['page'].nunique() if 'page' in df.columns else 'N/A'} unique types"
)

print("\nData Quality:")
missing_cols = missing_info[missing_info["Missing_Percentage"] > 0]
if not missing_cols.empty:
    print(f"   • Missing values found in {len(missing_cols)} columns")
    for _, row in missing_cols.iterrows():
        print(f"     - {row['Column']}: {row['Missing_Percentage']:.1f}% missing")
else:
    print("   • No missing values detected")

if duplicates > 0:
    print(f"   • Duplicate records: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")
else:
    print("   • No duplicate records found")

print("\nUser Behavior Insights:")
if "page" in df.columns:
    top_events = df["page"].value_counts().head(3)
    print(
        f"   • Top 3 events: {', '.join([f'{event} ({count})' for event, count in top_events.items()])}"
    )

if "level" in df.columns:
    level_split = df["level"].value_counts()
    print(f"   • Subscription levels: {dict(level_split)}")

if churn_events:
    print(f"   • Potential churn indicators found: {', '.join(churn_events)}")
else:
    print("   • No obvious churn events detected in page types")

print("\nKey Challenges Identified:")
challenges = []
if any(missing_info["Missing_Percentage"] > 10):
    challenges.append("High missing values in some columns")
if duplicates / len(df) > 0.01:
    challenges.append("Duplicate records present")
if not churn_events:
    challenges.append("Churn definition needs to be inferred from user behavior")

if challenges:
    for i, challenge in enumerate(challenges, 1):
        print(f"   {i}. {challenge}")
else:
    print("   No major data quality issues identified")

print("\nNext Steps:")
print("   1. Define churn criteria based on user inactivity patterns")
print("   2. Perform detailed churn analysis and user segmentation")
print("   3. Engineer features capturing user engagement trends")
print("   4. Address data quality issues before modeling")

## 9. Data Export for Further Analysis

In [ ]:
# Save processed data with additional features for next notebook
df_processed = df.copy()

# Export summary statistics
summary_stats = {
    "total_events": len(df),
    "unique_users": df["userId"].nunique() if "userId" in df.columns else 0,
    "unique_event_types": df["page"].nunique() if "page" in df.columns else 0,
    "date_range_days": (
        (df["datetime"].max() - df["datetime"].min()).days
        if "datetime" in df.columns
        else 0
    ),
    "potential_churn_events": churn_events,
    "data_quality_issues": challenges,
}

print("Initial data exploration completed!")
print("Summary statistics and processed data ready for churn definition analysis.")

In [ ]:
summary_stats